<a href="https://colab.research.google.com/github/Innovation-On-Blockchain/Data-Modeling/blob/main/Modeling_and_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TODO: Data Processing
# ==============================================================
# Input:  raw/transactions_raw.parquet, labels/labels.parquet
# Output: processed/formatted_transactions.parquet
#         processed/node_labels.parquet
#         processed/data_splits.json
# ==============================================================
from google.colab import drive

drive.mount('/content/drive')
import pandas as pd
import json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import pyarrow.parquet as pq
shared_folder_path = '/content/drive/MyDrive/Math_168_Folder'
os.chdir(shared_folder_path)

path = "/content/drive/MyDrive/Math_168_Folder/data/raw/transactions_raw.parquet"
print(f"File size: {os.path.getsize(path) / 1e9:.2f} GB")

meta = pq.read_metadata(path)
print(f"Rows: {meta.num_rows:,}")
print(f"Columns: {meta.num_columns}")

File size: 5.58 GB
Rows: 90,728,163
Columns: 5


In [ ]:
# ================================================================
# Load raw data
# ================================================================
# Read the two input parquet files produced by the Rust pipeline.
# transactions_raw columns: tx_hash, block_timestamp, from_address,
#                           to_address, value_wei  (all strings)
# labels columns: address, label, label_source, source_url, retrieved_at
# ================================================================

import pyarrow.parquet as pq

path = "/content/drive/MyDrive/Math_168_Folder/data/raw/transactions_raw.parquet"

chunks = []
parquet_file = pq.ParquetFile(path)

for batch in parquet_file.iter_batches(batch_size=1_000_000):
    chunk = batch.to_pandas()
    # Filter invalid rows immediately to save memory
    chunk = chunk[
        chunk["from_address"].notna() & (chunk["from_address"] != "") &
        chunk["to_address"].notna()   & (chunk["to_address"]   != "")
    ]
    chunks.append(chunk)
    print(f"  Processed batch — kept {len(chunk):,} rows", end="\r")

df = pd.concat(chunks, ignore_index=True)
del chunks  # free memory
import gc; gc.collect()

print(f"\nLoaded {len(df):,} valid rows")

labels = pd.read_parquet("/content/drive/MyDrive/Math_168_Folder/data/labels/labels.parquet")

print(f"Raw transactions loaded: {len(df):,} rows")
print(f"Labels loaded:           {len(labels):,} sanctioned addresses")
print()

# Quick data-quality report
n_empty_from = (df["from_address"].isna() | (df["from_address"] == "")).sum()
n_empty_to   = (df["to_address"].isna()   | (df["to_address"]   == "")).sum()
print(f"Empty/null from_address: {n_empty_from:,} / {len(df):,}")
print(f"Empty/null to_address:   {n_empty_to:,} / {len(df):,}")
print()


Loaded 90,728,163 valid rows
Raw transactions loaded: 90,728,163 rows
Labels loaded:           82 sanctioned addresses

Empty/null from_address: 0 / 90,728,163
Empty/null to_address:   0 / 90,728,163



In [ ]:
# This is already done in the previous part (data collection), so we can safely skip this.
# # ================================================================
# # Step 1: Filter Invalid Transactions
# # ================================================================
# # Remove rows where from_address or to_address is null or empty.
# # These are contract-creation transactions (no recipient), which
# # are not useful for money-flow graph analysis.
# # ================================================================

# before = len(df)

# df = df[
#     df["from_address"].notna() & (df["from_address"] != "") &
#     df["to_address"].notna()   & (df["to_address"]   != "")
# ]
# df = df.reset_index(drop=True)

# after = len(df)
# print(f"Step 1 — Filter invalid transactions")
# print(f"  Before: {before:,}  After: {after:,}  Removed: {before - after:,}")
# print()

# if after == 0:
#     print("WARNING: All rows were removed (every to_address is empty).")
#     print("The Rust pipeline likely has a bug. Fix the data and re-run.")
#     print("Skipping remaining steps.")
#     exit(0)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ================================================================
# Step 2: Create Address-to-ID Mapping
# ================================================================
# Collect every unique address from both from_address and to_address.
# Sort them so the mapping is deterministic, then assign each one
# a numeric ID starting from 0. Add from_id / to_id columns.
# ================================================================

all_addresses = pd.concat([df["from_address"], df["to_address"]]).unique()
address_to_id = {addr: i for i, addr in enumerate(sorted(all_addresses))}

df["from_id"] = df["from_address"].map(address_to_id)
df["to_id"]   = df["to_address"].map(address_to_id)

print(f"Step 2 — Address-to-ID mapping")
print(f"  Unique addresses: {len(address_to_id):,}")
print(f"  ID range: 0 .. {len(address_to_id) - 1}")
print()

Step 2 — Address-to-ID mapping
  Unique addresses: 33,895,535
  ID range: 0 .. 33895534



In [ ]:
# ================================================================
# Step 3: Convert Timestamps
# ================================================================
# Parse block_timestamp from ISO-8601 strings to datetime objects.
# Sort the dataframe by timestamp ascending (earliest first).
# Subtract the earliest timestamp so the first row has Timestamp=0
# and all other values are seconds elapsed since that first tx.
# ================================================================

df["block_timestamp"] = pd.to_datetime(df["block_timestamp"])
df = df.sort_values("block_timestamp").reset_index(drop=True)

t_min = df["block_timestamp"].min()
df["Timestamp"] = (df["block_timestamp"] - t_min).dt.total_seconds().astype(int)

print(f"Step 3 — Convert timestamps")
print(f"  Earliest: {t_min}")
print(f"  Latest:   {df['block_timestamp'].max()}")
print(f"  Span:     {df['Timestamp'].max():,} seconds")
print(f"  First Timestamp value: {df['Timestamp'].iloc[0]}")
print()

df.drop(columns=["block_timestamp"], inplace=True)
gc.collect()

Step 3 — Convert timestamps
  Earliest: 2015-08-07 05:01:09+00:00
  Latest:   2026-02-19 19:21:47+00:00
  Span:     332,605,238 seconds
  First Timestamp value: 0



17

In [ ]:
# ================================================================
# Step 4: Convert Values (Wei -> ETH)
# ================================================================
# value_wei is a string in wei (1 ETH = 1e18 wei).
# Convert to float ETH. Store as both "Amount Sent" and
# "Amount Received" — they are equal for direct ETH transfers.
# ================================================================

WEI_PER_ETH = 10**18

df["Amount Sent"]     = df["value_wei"].astype(float) / WEI_PER_ETH
df["Amount Received"] = df["Amount Sent"]

print(f"Step 4 — Wei to ETH conversion")
print(f"  Min:  {df['Amount Sent'].min():.8f} ETH")
print(f"  Max:  {df['Amount Sent'].max():.8f} ETH")
print(f"  Mean: {df['Amount Sent'].mean():.8f} ETH")
print()

df.drop(columns=["value_wei"], inplace=True)
gc.collect()


Step 4 — Wei to ETH conversion
  Min:  0.00000000 ETH
  Max:  1400000.00000000 ETH
  Mean: 14.01213088 ETH



7

In [ ]:
# ================================================================
# Step 5: Add Edge Labels
# ================================================================
# Build a set of the 82 OFAC-sanctioned addresses from labels.
# For each transaction edge:
#   - Is Laundering = 1  if from_address OR to_address is sanctioned
#   - Is Laundering = 0  if NEITHER endpoint is sanctioned
# Edges between two clean addresses are intentionally kept as 0 —
# they give the GNN examples of normal graph structure.
# ================================================================

sanctioned_set = set(labels["address"])

df["Is Laundering"] = (
    df["from_address"].isin(sanctioned_set) |
    df["to_address"].isin(sanctioned_set)
).astype(int)

n_launder = (df["Is Laundering"] == 1).sum()
n_clean   = (df["Is Laundering"] == 0).sum()
print(f"Step 5 — Edge labels")
print(f"  Sanctioned addresses: {len(sanctioned_set)}")
print(f"  Laundering edges (1): {n_launder:,}")
print(f"  Clean edges (0):      {n_clean:,}")
print()

df.drop(columns=["from_address", "to_address"], inplace=True)
gc.collect()

Step 5 — Edge labels
  Sanctioned addresses: 82
  Laundering edges (1): 10,453
  Clean edges (0):      90,717,710



7

In [ ]:
# ================================================================
# Step 6: Build Formatted Transactions File
# ================================================================
# TODO: Assemble the final dataframe with columns in this order:
#   EdgeID, from_id, to_id, Timestamp, Amount Sent, Sent Currency,
#   Amount Received, Received Currency, Payment Format, Is Laundering
#
# - EdgeID = row index starting at 0
# - Sent Currency, Received Currency, Payment Format = always 1
#
# Save to: processed/formatted_transactions.parquet
# ================================================================


SENT_CURRENCY_CODE = 1
RECEIVED_CURRENCY_CODE = 1
PAYMENT_FORMAT_CODE = 1

formatted = pd.DataFrame({
    "EdgeID":            range(len(df)),
    "from_id":           df["from_id"].astype(int),
    "to_id":             df["to_id"].astype(int),
    "Timestamp":         df["Timestamp"].astype(int),
    "Amount Sent":       df["Amount Sent"].astype(float),
    "Sent Currency":     SENT_CURRENCY_CODE,
    "Amount Received":   df["Amount Received"].astype(float),
    "Received Currency": RECEIVED_CURRENCY_CODE,
    "Payment Format":    PAYMENT_FORMAT_CODE,
    "Is Laundering":     df["Is Laundering"].astype(int),
})

# Ensure the processed folder exists
processed_dir = "/content/drive/MyDrive/Math_168_Folder/data/processed"
os.makedirs(processed_dir, exist_ok=True)

formatted_path = os.path.join(processed_dir, "formatted_transactions.parquet")
formatted.to_parquet(formatted_path, index=False)

print("Step 6 — Formatted transactions saved")
print(f"  Path: {formatted_path}")
print(f"  Rows: {len(formatted):,}")
print(f"  Columns: {list(formatted.columns)}")
print()

del df, formatted
gc.collect()
print("Memory freed before Step 7")

Step 6 — Formatted transactions saved
  Path: /content/drive/MyDrive/Math_168_Folder/data/processed/formatted_transactions.parquet
  Rows: 90,728,163
  Columns: ['EdgeID', 'from_id', 'to_id', 'Timestamp', 'Amount Sent', 'Sent Currency', 'Amount Received', 'Received Currency', 'Payment Format', 'Is Laundering']

Memory freed before Step 7


In [ ]:
# ================================================================
# Step 7: Build Node Labels File
# ================================================================
# TODO: For each address in address_to_id:
#   - node_id = address_to_id[address]
#   - is_sanctioned = 1 if address in sanctioned_set, else 0
#
# Columns: node_id, is_sanctioned
# Save to: processed/node_labels.parquet
# ================================================================

node_labels = pd.DataFrame({
    "node_id": list(address_to_id.values()),
    "is_sanctioned": [1 if addr in sanctioned_set else 0 for addr in address_to_id.keys()],
}).sort_values("node_id").reset_index(drop=True)

node_labels_path = os.path.join(processed_dir, "node_labels.parquet")
node_labels.to_parquet(node_labels_path, index=False)

print("Step 7 — Node labels saved")
print(f"  Path: {node_labels_path}")
print(f"  Rows (nodes): {len(node_labels):,}")
print(f"  Sanctioned nodes: {node_labels['is_sanctioned'].sum():,}")
print()


Step 7 — Node labels saved
  Path: /content/drive/MyDrive/Math_168_Folder/data/processed/node_labels.parquet
  Rows (nodes): 33,895,535
  Sanctioned nodes: 75



In [ ]:
# ================================================================
# Step 8: Create Train/Validation Split
# ================================================================
# TODO: Data is sorted by time (Step 3).
#   - split_point = int(len(formatted) * 0.8)
#   - train_edge_ids = list(range(0, split_point))
#   - val_edge_ids   = list(range(split_point, len(formatted)))
#
# Save as processed/data_splits.json:
#   {"train_edge_ids": [...], "val_edge_ids": [...]}
# ================================================================
n_edges = 90_728_163
split_point = int(n_edges * 0.8)
train_edge_ids = list(range(0, split_point))
val_edge_ids   = list(range(split_point, n_edges))

splits = {
    "train_edge_ids": train_edge_ids,
    "val_edge_ids": val_edge_ids,
}

splits_path = os.path.join(processed_dir, "data_splits.json")
with open(splits_path, "w") as f:
    json.dump(splits, f)

print("Step 8 — Data splits saved")
print(f"  Path: {splits_path}")
print(f"  Train edges: {len(train_edge_ids):,}")
print(f"  Val edges:   {len(val_edge_ids):,}")
print()


Step 8 — Data splits saved
  Path: /content/drive/MyDrive/Math_168_Folder/data/processed/data_splits.json
  Train edges: 72,582,530
  Val edges:   18,145,633

